# Исследование надежности заемщиков

<div class="alert alert-success">
    <b>ОПИСАНИЕ ПРОЕКТА:</b>
    
Заказчик проекта - кредитный отдел банка. Для построения модели кредитного скоринга необходимо выявить зависимость платежоспособности (оплата кредита в установленный срок) клиентов от семейного положения, количества детей, целей кредита и уровня дохода.</div>


<div class="alert alert-success">
    <b>ЦЕЛЬ ПРОЕКТА:</b>

1. Установить зависимость между наличием детей и возвратом кредита в срок.
2. Установить  зависимость между семейным положением и возвратом кредита в срок.
3. Установить зависимость между уровнем дохода и возвратом кредита в срок.
4. Установить как разные цели кредита влияют на его возврат в срок.</div>


## Откройте таблицу и изучите общую информацию о данных

**Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Выведите первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

*У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставbv их как есть.*

**Выведите перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

**Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
data.isna().sum()

children               0
days_employed       2162
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Изменение типов данных

**Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [12]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру.**

In [13]:
data['education'] = data['education'].str.lower()

**Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [14]:
data.duplicated().sum()

71

In [15]:
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [16]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [17]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [18]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [19]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [20]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследуйте данные и ответьте на вопросы

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

**Создадим функцию для категоризации людей по количеству детей:**
- `семьи с детьми (1 и 2 ребенка)`,
- `многодетные семьи (3 ребенка и более)`,
- `бездетные`.


In [21]:
def children_category(children):
    try:
        if 1 <= children <= 2:
            return 'семьи с детьми'
        elif children >= 3:
            return 'многодетные семьи'
        return 'бездетные'
    except:
        pass

**Cоздадим столбец** `'num_children'`

In [22]:
data['num_children'] = data['children'].apply(children_category)

**Cгруппируем датафрейм по столбцу** `'num_children'`  
**агрегируем функции количества, суммы, среднего к столбцу** `'debt'` 

In [23]:
deb_num_children = data.groupby('num_children')['debt'].agg(['count',
                                                         'sum', 
                                                         'mean'])

**Переведем значения в столбце** `'mean'` **в проценты**  

In [24]:
deb_num_children['mean'] = deb_num_children['mean'].apply('{:.2%}'.format)

**Переименуем столбцы для лучшей визуализации**  

In [25]:
deb_num_children = deb_num_children.rename(columns= {'count':'Заемщики', 
                                                     'sum':'Дебиторы', 
                                                     'mean':'Процент_дебиторов'
                                                    })

**Отсортируем данные по столбцу** `'Процент_дебиторов'` **по убыванию**   

In [26]:
deb_num_children.sort_values(by='Процент_дебиторов', ascending=False)

,Заемщики,Дебиторы,Процент_дебиторов
num_children,,,
семьи с детьми,6860,638,9.30%
многодетные семьи,380,31,8.16%
бездетные,14091,1063,7.54%


<div class="alert alert-info">
    <b>Вывод:</b> 
    
По категории <b>"многодетные семьи"</b> выборка на порядок меньше относительно категорий <b>"семьи с одним или двумя детьми"</b> и <b>"бездетные"</b>. Поэтому выводов по категории **"многодетные семьи"** делать не будем. Сравнивая выборки по категориям **"семьи с одним или двумя детьми"** и **"бездетные"** делаем вывод, что **наличие детей** у клиента увеличивает вероятность возникновения задолженности по сравнению с **отсутствием детей** : **9.30%** против **7.54%** (относительная разница порядка **25%**).    
</div>

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

**Cгруппируем датафрейм по столбцу** `'family_status'`  
**агрегируем функции количества, суммы, среднего к столбцу** `'debt'` 

In [27]:
deb_family_status = data.groupby('family_status')['debt'].agg(['count', 
                                                               'sum', 
                                                               'mean'
                                                              ])

**Переведем значения в столбце** `'mean'` **в проценты**  

In [28]:
deb_family_status['mean'] = deb_family_status['mean'].apply('{:.2%}'.format)

**Переименуем столбцы для лучшей визуализации**  

In [29]:
deb_family_status = deb_family_status.rename(columns={'count':'Заемщики',
                                                      'sum':'Дебиторы',
                                                      'mean':'Процент_дебиторов'})

**Отсортируем данные по столбцу** `'Процент_дебиторов'` **по убыванию**   

In [30]:
deb_family_status.sort_values(by='Процент_дебиторов', ascending=False)

,Заемщики,Дебиторы,Процент_дебиторов
family_status,,,
Не женат / не замужем,2796,273,9.76%
гражданский брак,4134,385,9.31%
женат / замужем,12261,927,7.56%
в разводе,1189,84,7.06%
вдовец / вдова,951,63,6.62%


<div class="alert alert-info">
    <b>Вывод:</b> 

Зависимость по возврату кредита в срок от семейного положения прослеживается следующая: заёмщики, которые состоят или состояли в официальном браке (категории **'женат / замужем'**, **'в разводе' и 'вдовец/вдова'**) имеют большую вероятность своевременного возврата кредита, чем заемщики (категории **'не женат / не замужем' и 'гражданский брак'**), которые в официальном браке не состоят.    


#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

**Cгруппируем датафрейм по столбцу** `'total_income_category'`  
**агрегируем функции количества, суммы, среднего к столбцу** `'debt'` 

In [31]:
deb_total_income = data.groupby('total_income_category')['debt'].agg(['count', 
                                                               'sum', 
                                                               'mean'
                                                              ])

**Переведем значения в столбце** `'mean'` **в проценты**  

In [32]:
deb_total_income['mean'] = deb_total_income['mean'].apply('{:.2%}'.format)

**Переименуем столбцы для лучшей визуализации**  

In [33]:
deb_total_income = deb_total_income.rename(columns={'count':'Заемщики',
                                                      'sum':'Дебиторы',
                                                      'mean':'Процент_дебиторов'})

**Отсортируем данные по столбцу** `'Процент_дебиторов'` **по убыванию**   

In [34]:
deb_total_income.sort_values(by='Процент_дебиторов', ascending=False)

,Заемщики,Дебиторы,Процент_дебиторов
total_income_category,,,
E,22,2,9.09%
C,15921,1353,8.50%
A,25,2,8.00%
B,5014,354,7.06%
D,349,21,6.02%


#### Как разные цели кредита влияют на его возврат в срок?

**Cгруппируем датафрейм по столбцу** `'purpose_category'`  
**агрегируем функции количества, суммы, среднего к столбцу** `'debt'` 

In [35]:
deb_purpose_category = data.groupby('purpose_category')['debt'].agg(['count', 
                                                               'sum', 
                                                               'mean'
                                                              ])

**Переведем значения в столбце** `'mean'` **в проценты**  

In [36]:
deb_purpose_category['mean'] = deb_purpose_category['mean'].apply('{:.2%}'.format)

**Переименуем столбцы для лучшей визуализации**  

In [37]:
deb_purpose_category = deb_purpose_category.rename(columns={'count':'Заемщики',
                                                      'sum':'Дебиторы',
                                                      'mean':'Процент_дебиторов'})

**Отсортируем данные по столбцу** `'Процент_дебиторов'` **по убыванию**   

In [38]:
deb_purpose_category.sort_values(by='Процент_дебиторов', ascending=False)

,Заемщики,Дебиторы,Процент_дебиторов
purpose_category,,,
операции с автомобилем,4279,400,9.35%
получение образования,3988,369,9.25%
проведение свадьбы,2313,183,7.91%
операции с недвижимостью,10751,780,7.26%


<div class="alert alert-info">
    <b>Вывод:</b> 

- **Группа риска** по возврату кредита в срок - это **категории 'операции с автомобилем' и 'получение образования	'**. Причина предположительно, что клиенты считают автомобиль или образование не средство первой необходимости, как, например, залоговая квартира по ипотеке. 
- Категория **'проведение свадьбы'** неплохой процент дебиторки. Причина, скорее всего не в слишком больших суммах кредита на проведение свадьбы, а также возможно связано с определенной окупаемостью свадьбы.  
- **'Операции с недвижимостью'** категория с **наименьшим риском** возникновения просроченной задолженности. Видимо заемщики наиболее ответственно подходят к любым операциям с недвижимостью

#### Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* Причинами могут быть:
- человеческий фактор (неверный ввод данных, подлог данных о клиенте);
- техногенный фактор (технические проблемы, природные катаклизмы).


#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Так как медиана выбросоустойчива (робастна) относительно среднего значения. То есть по сути медиана - это страховка от выбросов и аномальных значений.

### Oбщий вывод.

В ходе исследования надёжности заемщиков были изучены полученные  от банка данные, предообработаны (заполнены пропуски, обработаны аномальные значения, удалены дубликаты, оптимизированы некоторые категории данных). Заёмщики были сгруппированы по различным параметрам, на основании группировок были выявлены определенные закономерности. По итогам проделанной работы по поставленным перед нами вопросам можно сделать следующие выводы:

*Зависимость между количеством детей и возвратом кредита в срок*
- **Группа риска** по возврату кредита в срок - это **семьи с одним или двумя детьми** . (процент дебиторов **9.30%**)
- **Бездетные** являются группой с **наименьшим риском** возникновения просроченной задолженности. (процент дебиторов **7.54%**)
<div class="alert alert-info">- То есть <b>наличие детей</b> увеличивает вероятность возникновения задолженности по сравнению с <b>отсутствием детей</b>.

*Зависимость между семейным положением и возвратом кредита в срок*
- **Группа риска** по возврату кредита в срок - это **'не женатые' и 'состоящие в гражданском браке'**. (процент дебиторов соответственно **9.76%** и **9.31%**)
- **'В разводе' и 'вдовцы'** являются группами с **наименьшим риском** возникновения просроченной задолженности (процент дебиторов соответственно **7.06%** и **6.62%**)
<div class="alert alert-info">- Делаем вывод, что заёмщики, которые состоят или состояли в официальном браке имеют большую вероятность своевременного возврата кредита, чем заёмщики , которые в официальном браке не состоят.  

*Зависимость между уровнем дохода и возвратом кредита в срок*
- **Группа риска** по возврату кредита в срок - это **заёмщики со средним уровнем дохода** - заёмщики со средним уровнем дохода. (процент дебиторов **8.50%**)
- Категория **'заемщики с высоким уровнем дохода'** с меньшей вероятностью будут иметь просрочку по платежу. (процент дебиторов соответственно **7.06%**)
<div class="alert alert-info">-По уровню дохода делаем вывод, чем выше доход заёмщика тем ниже вероятность возникновения просроченной задолженности.

*Зависимость возврата кредита в срок от целей кредита*
- **Группа риска** по возврату кредита в срок - это категории **'операции с автомобилем' и 'получение образования'** (процент дебиторов соответственно **9.35%** и **9.25%**)
- **'Операции с недвижимостью'** категория с **наименьшим риском** возникновения дебиторской задолженности (процент дебиторов соответственно **7.91%** и **7.26%**)
